#### 문제 설명  
N x N 크기인 정사각 격자 형태의 지형이 있습니다. 각 격자 칸은 1 x 1 크기이며, 숫자가 하나씩 적혀있습니다. 격자 칸에 적힌 숫자는 그 칸의 높이를 나타냅니다.  
  
이 지형의 아무 칸에서나 출발해 모든 칸을 방문하는 탐험을 떠나려 합니다. 칸을 이동할 때는 상, 하, 좌, 우로 한 칸씩 이동할 수 있는데, 현재 칸과 이동하려는 칸의 높이 차가 height 이하여야 합니다. 높이 차가 height 보다 많이 나는 경우에는 사다리를 설치해서 이동할 수 있습니다. 이때, 사다리를 설치하는데 두 격자 칸의 높이차만큼 비용이 듭니다. 따라서, 최대한 적은 비용이 들도록 사다리를 설치해서 모든 칸으로 이동 가능하도록 해야 합니다. 설치할 수 있는 사다리 개수에 제한은 없으며, 설치한 사다리는 철거하지 않습니다.  
  
각 격자칸의 높이가 담긴 2차원 배열 land와 이동 가능한 최대 높이차 height가 매개변수로 주어질 때, 모든 칸을 방문하기 위해 필요한 사다리 설치 비용의 최솟값을 return 하도록 solution 함수를 완성해주세요.  
   
#### 제한 조건  
- land는 N x N크기인 2차원 배열입니다.
- land의 최소 크기는 4 x 4, 최대 크기는 300 x 300입니다.
- land의 원소는 각 격자 칸의 높이를 나타냅니다.
- 격자 칸의 높이는 1 이상 10,000 이하인 자연수입니다.
- height는 1 이상 10,000 이하인 자연수입니다.  
  
#### 입출력 예  
|land|	height|	result|  
|:---:|:---:|:---:|  
|[[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]|	3|	15|  
|[[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]|	1|	18|  

---

### 문제 풀이

In [1]:
from queue import Queue
from collections import defaultdict
import math

def get_ladder(N, visited, land, idx): # 각 그룹 간의 최소 거리 사다리를 구하는 함수
    ladder_dict = defaultdict(lambda: math.inf) # 사다리가 연결하는 두 그룹을 key로 하고, 사다리의 설치 비용을 value로 하는 dict
    checked = [[False for _ in range(N)] for _ in range(N)] # 해당 좌표 체크 여부 확인
    direction = [(1, 0), (-1, 0), (0, 1), (0, -1)] # 인접한 네 방향
    for y in range(N):
        for x in range(N): # 모든 좌표에 대해서
            checked[y][x] = True # 현재 좌표를 탐색함을 체크
            for add_y, add_x in direction:
                if 0 <= add_y+y < N and 0 <= add_x+x < N and not checked[add_y+y][add_x+x] and visited[y][x] != visited[add_y+y][add_x+x]:
                # 아직 탐색하지 않은 인접한 방향에서 서로 다른 그룹이 존재한다면
                    a, b = sorted((visited[y][x], visited[add_y+y][add_x+x])) # 서로 인접한 그룹의 인덱스 a, b
                    ladder_dict[(a,b)] = min(ladder_dict[(a,b)], abs(land[y][x] - land[add_y+y][add_x+x]))
                    # (a, b) 그룹 사이의 최소 사다리 설치 비용을 구한다
    ladders = [] # 사다리 정보
    for k, v in ladder_dict.items(): # 위에서 구한 모든 사다리에 대해서
        a, b = k
        ladders.append((a, b, v)) # (그룹 a, 그룹 b, 사다리 설치 비용) 형태로 저장
    ladders.sort(key=lambda x: x[2]) # 그리고 사다리 설치 비용 오름차순으로 정렬
    return ladders # 사다리 정보 반환

def find_parent(x, parent): # 각 부분집합의 루트를 찾는 함수
    if x == parent[x]: # 현재 탐색하는 그룹이 루트라면 바로 반환
        return x
    else:
        p = find_parent(parent[x], parent) # 재귀적으로 루트를 찾는다.
        parent[x] = p # x그룹의 루트 갱신
        return p
    
def union_find(x, y, parent): # disjoint set(상호배타적인 부분집합)을 합치는 함수
    x = find_parent(x, parent)
    y = find_parent(y, parent)
    parent[y] = x # y 그룹을 x 그룹에 합친다.

def get_min(visited, ladders, idx): # 각 지형을 사다리로 연결했을 때 나오는 최소값을 구하는 함수
    answer = 0 # 답
    parent = {i:i for i in range(idx)} # 각 그룹별 자신의 루트 그룹
    for a, b, val in ladders: # 모든 사다리 정보에 대해서
        if find_parent(a, parent) != find_parent(b, parent): # 만약 그룹 a와 그룹 b가 아직 연결되어 있지 않을 때 
            union_find(a, b, parent) # a와 b를 합친다
            answer += val # 그리고 사다리 설치 비용을 합한다.
        
        if len(parent.values()) == 1: # 모든 그룹이 합쳤을 때 루프를 빠져나간다
            break
    return answer # 답 반환

def go_without_ladder(N, visited, land, height, s_y, s_x, idx): # idx번 그룹 내의 지형을 찾아내서 표시하는 함수
    direction = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    q = Queue() # 큐
    q.put((s_y, s_x)) # 현재 좌표 정보를 큐에 push
    
    while not q.empty():
        y, x = q.get()
        for add_y, add_x in direction: # 인접한 네 방향에 아직 탐색하지 않은 공간이 있다면
            if 0 <= add_y+y < N and 0 <= add_x+x < N and visited[add_y+y][add_x+x] == -1:
                if abs(land[y][x]-land[add_y+y][add_x+x]) <= height:
                # 사다리 없이 해당 좌표로 이동할 수 있을 경우
                    visited[add_y+y][add_x+x] = idx # 같은 그룹 번호 저장
                    q.put((add_y+y, add_x+x)) # 다시 큐에 좌표 push
    return

def solution(land, height):
    N = len(land)  # 지형의 높이 및 너비
    visited = [[-1 for _ in range(N)] for _ in range(N)] # 위치별 각 그룹의 인덱스를 저장할 것임
    idx = 0 # 그룹 인덱스
    for y in range(N):
        for x in range(N): # 지형 내의 모든 좌표에 대해서
            if visited[y][x] != -1: # 이미 탐색한 그룹 내의 좌표는 제외하고
                continue
            visited[y][x] = idx
            go_without_ladder(N, visited, land, height, y, x, idx)
            # 현재 탐색중인 그룹 내의 영토를 모두 해당 그룹 인덱스로 표시
            idx += 1 # 그룹 인덱스 1 증가
    ladders = get_ladder(N, visited, land, idx) # 그룹 간의 최소 비용 사다리를 구한다
    answer = get_min(visited, ladders, idx) # 모든 그룹을 연결할 수 있는 최소 비용을 구한다
    return answer # 답 반환

In [2]:
land = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
solution(land, height)

15

In [3]:
land = [[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]
height = 1
solution(land, height)

18